In [1]:
#import MongoClient 
from pymongo import MongoClient 
from pprint import pprint
from random import randint
import pandas as pd

In [3]:
#Step 1: Connect to MongoDB - Note: Change connection string as needed
client = MongoClient(port=27017)
db=client.Organizer 

In [4]:
# step 2: create collections/ tables
collection=db.Student
collection=db.City
collection=db.County
collection=db.Courses

In [5]:
#step 3: Insert the records in the connections  
db.Student.insert_one({"First_Name":"Jacob","Last_Name":"Odeny","Email":"jacobodeny@gmail.com","telphone":"0575174540","Physical_Address":"Kano","Gender":"Male", "City_Code":"1", "Country_Code":"1", "CourseID":"2", "StudentID":"1"})
db.Student.insert_one({"First_Name":"Olal","Last_Name":"Deric","Email":"dolal@gmail.com","telphone":"0705189701","Physical_Address":"Kisumu","Gender":"Male", "City_Code":"1", "Country_Code":"1", "CourseID":"3", "StudentID":"2"})
db.Student.insert_one({"First_Name":"Steve","Last_Name":"Misiga","Email":"oredo@gmail.com","telphone":"0705758502","Physical_Address":"Nairobi","Gender":"Male", "City_Code":"2", "Country_Code":"2", "CourseID":"2", "StudentID":"3"})
db.Student.insert_one({"First_Name":"Esther","Last_Name":"Mwangi","Email":"ess@gmail.com","telphone":"0806157590","Physical_Address":"Muranga","Gender":"Female", "City_Code":"3", "Country_Code":"3", "CourseID":"1", "StudentID":"4"})
db.Student.insert_one({"First_Name":"Raila","Last_Name":"Amolo","Email":"rais@yahoo.com","telphone":"0305158509","Physical_Address":"Bondo","Gender":"Male", "City_Code":"3", "Country_Code":"3", "CourseID":"4", "StudentID":"5"})
print("Records saved successfully")

Records saved successfully


In [7]:
cities={"City_Code":"1", 'city_name':'Nairobi'}, {"City_Code":"2",'city_name':'Kisumu'}, {"City_Code":"3",'city_name':'Mombasa'}
db.City.insert_many(cities)
print('Records saved successfully')

Records saved successfully


In [9]:
counties={"County_Code":"1", 'County_Name':'Nairobi'}, {"County_Code":"2", 'County_Name':'Kisumu'}, {"County_Code":"3",'County_Name':'Mombasa'}
db.County.insert_many(counties)
print('Records  saved successfully')

Records  saved successfully


In [10]:
course={"CourseID":"1", 'Course_Name':'Statistics'}, {"CourseID":"2", 'Course_Name':'Medicine'}, {"CourseID":"3", 'Course_Name':'Law'}, {"CourseID":"4", 'Course_Name':'IT'}, {"CourseID":"5",'Course_Name':'BeCoM'}
db.Courses.insert_many(course)
print('Records saved successfully')

Records saved successfully


In [11]:
#steep 4: create for every  dataframe
student_df=pd.DataFrame ([x for x in db.Student.find()])
student_df.set_index('StudentID', inplace=True)
student_df.drop('_id', inplace = True, axis=1)
student_df

,First_Name,Last_Name,Email,telphone,Physical_Address,Gender,City_Code,Country_Code,CourseID
StudentID,,,,,,,,,
1,Jacob,Odeny,jacobodeny@gmail.com,0575174540,Kano,Male,1,1,2
2,Olal,Deric,dolal@gmail.com,0705189701,Kisumu,Male,1,1,3
3,Steve,Misiga,oredo@gmail.com,0705758502,Nairobi,Male,2,2,2
4,Esther,Mwangi,ess@gmail.com,0806157590,Muranga,Female,3,3,1
5,Raila,Amolo,rais@yahoo.com,0305158509,Bondo,Male,3,3,4


In [12]:
#create a City dataframe
city=pd.DataFrame ([x for x in db.City.find()])
city.set_index("City_Code", inplace = True)
city.drop('_id', inplace = True, axis=1)
city

,city_name
City_Code,
1,Nairobi
2,Kisumu
3,Mombasa


In [14]:
#create the country dataframe
county=pd.DataFrame ([x for x in db.County.find()])
county.set_index("County_Code", inplace= True)
county.drop('_id', inplace= True, axis=1)
county

,County_Name
County_Code,
1,Nairobi
2,Kisumu
3,Mombasa


In [15]:
#Format the data retieved from the Courses collection using pandas
coursesdf=pd.DataFrame ([x for x in db.Courses.find()])
coursesdf.set_index("CourseID", inplace=True)
coursesdf.drop("_id", inplace = True, axis=1)
coursesdf

,Course_Name
CourseID,
1,Statistics
2,Medicine
3,Law
4,IT
5,BeCoM


In [16]:
#step 5: merge the sets 

join_student_courses= db.Student.aggregate(
[
    {
        "$lookup":{
            "from":"Courses",
            "localField":"CourseID",
            "foreignField":"CourseID",
            "as":"Student_Course"
        }
    },
    {"$unwind":"$Student_Course" },
    {
        "$project":{
            "_id":0,
            "StudentID":1,
            "First_Name":2,
            "Last_Name":3,
            "Gender":4,
            "Course_Name":"$Student_Course.Course_Name"
            
        }
    }
])
    
#present the data above in tabular format
studentcoursesdf=pd.DataFrame (list(join_student_courses))
#index
studentcoursesdf.set_index("StudentID", inplace= True)
#table
studentcoursesdf

,First_Name,Last_Name,Gender,Course_Name
StudentID,,,,
1,Jacob,Odeny,Male,Medicine
2,Olal,Deric,Male,Law
3,Steve,Misiga,Male,Medicine
4,Esther,Mwangi,Female,Statistics
5,Raila,Amolo,Male,IT
